# Sentiment analysis with Vader

Similar to other task in this project, the task of sentiment analysis is done per single dataset.

In [ ]:
from datetime import date
import os
import sys
import polars as pl

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import config
from paths import Paths
from src.sentiment_analysis import get_compound_parallel

channel_paths = Paths(channel_handle=config.channel_handle, date_obj=date(2025, 9, 6))

### Loading single dataset in polars

In [49]:
comments = pl.read_parquet(channel_paths.clean_comments_file_path, columns=["comment"])["comment"].to_list()
print(f"{len(comments):_} total comments lodaded.")

465_750 total comments lodaded.


### Running parallel task across multiple processors

In [ ]:
cpu_count = 8
compound_scores = get_compound_parallel(comments, cpu_count)

  0%|          | 0/465750 [00:00<?, ?it/s]

### Add column and save back to parquet

In [51]:
# Read
df = pl.read_parquet(channel_paths.clean_comments_file_path)

# Add column
df = df.with_columns(pl.Series("sentiment_score", compound_scores))

In [52]:
# Write
df.write_parquet(channel_paths.clean_comments_file_path)